MovieLens 1M Data Set
#Measuring rating disagreement by gender

In [50]:
import numpy as np
import pandas as pd
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_csv('dataset/movielens/users.dat', sep='::', header=None, names=unames)
users

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('dataset/movielens/ratings.dat', sep='::', header=None, names=rnames)
ratings

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('dataset/movielens/movies.dat', sep='::', header=None, names=mnames)
movies

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if sys.path[0] == '':


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [51]:
users[:5]
users.shape
ratings[:5]
ratings.shape

(1000209, 4)

In [52]:
x = ratings.groupby('user_id').size()
x.index[x < 20][:5]         #Verifying Each user has at least 20 ratings

y = ratings.groupby('user_id')
y.size()
y.groups
y.count()
y[['rating']].count()[:5]
y.rating.sum()[:5]
y.rating.mean()[:5]

z = ratings.groupby(['user_id', 'movie_id'])
z.size()[:5]


user_id  movie_id
1        1           1
         48          1
         150         1
         260         1
         527         1
dtype: int64

In [53]:
movies[:5]
movies.shape


(3883, 3)

In [54]:
'''
Analyzing the data spread across three tables is not a simple task; 
suppose wanted to compute mean ratings for a particular movie
by sex and age. this is much easier to do with all of the data merged
together into a single table
'''
merge_rating_user = pd.merge(ratings, users)
merge_rating_user[:5]
df = pd.merge(merge_rating_user, movies)
df.head(5)
df.iloc[0]

user_id                                            1
movie_id                                        1193
rating                                             5
timestamp                                  978300760
gender                                             F
age                                                1
occupation                                        10
zip                                            48067
title         One Flew Over the Cuckoo's Nest (1975)
genres                                         Drama
Name: 0, dtype: object

In [55]:
df.groupby(['title','gender']).size()
df.groupby(['title','gender', 'rating']).size()
'''
mean movie ratings for each film grouped by gender
'''
mean_ratings_by_gender = df.pivot_table(values='rating',
    index=['title'], columns=['gender'], aggfunc='mean')
mean_ratings_by_gender[:5]
'''
filter down to movies that received at least 250 ratings
'''
ratings_by_title = df.groupby('title').size()   #group the data by title and 
                    #use size() to get a Series of group sizes for each title
ratings_by_title[:5]
active_titles = ratings_by_title.index[ratings_by_title >= 250]
active_titles[:5]

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)',
       '12 Angry Men (1957)'],
      dtype='object', name='title')

In [56]:
mean_ratings_by_active_titles = mean_ratings_by_gender.loc[active_titles]  
                                    #index of titleshaving at least 250 ratings 
                                    #used to select rows from mean_ratings_by_gender
top_female_ratings = mean_ratings_by_active_titles.sort_index(by='F', ascending=False)
top_female_ratings[:5]

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  after removing the cwd from sys.path.


gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415


In [57]:
#mean_ratings.sort_index(by='F', ascending=False)
#mean_ratings_by_gender.sort_index(by=['F', 'title'], ascending=False).sort_values('title')[:5]
mean_ratings_by_gender.sort_index(by=['F', 'title'], ascending=False)[:5]
x = mean_ratings_by_gender.index[mean_ratings_by_gender['F'] == 5]
mean_ratings_by_gender.loc[x][:5]

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  This is separate from the ipykernel package so we can avoid doing imports until


gender,F,M
title,,
24 7: Twenty Four Seven (1997),5.0,3.750000
Ayn Rand: A Sense of Life (1997),5.0,4.000000
"Ballad of Narayama, The (Narayama Bushiko) (1958)",5.0,3.428571
Battling Butler (1926),5.0,3.222222
Belly (1998),5.0,3.000000


In [58]:
'''
find the movies that are most divisive between male and female viewers. 
One way is to add a column to mean_ratings containing the difference in means, 
then sort by that
'''
mean_ratings_by_active_titles['diff'] = mean_ratings_by_active_titles['M'] - mean_ratings_by_active_titles['F']
sorted_by_diff = mean_ratings_by_active_titles.sort_index(by = 'diff')
sorted_by_diff[:15]
'''
Reversing the order of the rows and again slicing off the top 15 rows, 
the movies preferred by men that women didn’t rate as highly
'''
sorted_by_diff[::-1][:15]       #Sorted order reversely

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  import sys


gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


In [59]:
'''
movies that elicited the most disagreement among viewers, independent of gender. 
Disagreement can be measured by the variance or standard deviation of the ratings
'''
std_rating_by_title = df.groupby('title')['rating'].std()
std_rating_by_title         #Standard deviation of rating grouped by title
std_rating_by_title = std_rating_by_title.loc[active_titles] #Filter to Active titles
std_rating_by_title
std_rating_by_title.sort_values(ascending=False)[:10]
                            #Order Series by value in descending order
                            

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64

In [60]:
'''
List of geners containes '|'
'''
x = [pipe for pipe in df['genres'] if '|' in pipe]
type(x)
set(x)                      #Unique Values

df[df['genres'].isin(['Action', 'Drama'])]      #Matched values in col
df[df.genres.isin(['Action', 'Drama'])][:5]
df[~df.genres.isin(['Action', 'Drama'])][:5]    #not matched
df[df.genres.str.contains("Action|Drama", regex=True)][:12]
df[df.genres.str.contains("|", regex=False)].loc[:,'user_id':'title'][:5]
df[df.genres.str.contains("|", regex=False)].loc[:,['user_id','title', 'ratings', 'genres']][:5]
df[df.genres.str.contains("|", regex=False)].loc[:,['user_id','title', 'rating', 'genres']][:5]


,user_id,title,rating,genres
1725,1,James and the Giant Peach (1996),3,Animation|Children's|Musical
1726,23,James and the Giant Peach (1996),2,Animation|Children's|Musical
1727,49,James and the Giant Peach (1996),3,Animation|Children's|Musical
1728,53,James and the Giant Peach (1996),5,Animation|Children's|Musical
1729,57,James and the Giant Peach (1996),4,Animation|Children's|Musical


In [61]:
df.iloc[:, 0:2][:5]
df.iloc[:, [0,-2]][:5]
df.iloc[:, 3:-5][:5]


,timestamp,gender
0,978300760,F
1,978298413,M
2,978220179,M
3,978199279,M
4,978158471,M
